# 14 Cancer Microarray
**PAGE 654.** Ramaswamy et al. (2001) present a difficult microarray classification problem, involving a training set of 144 patients with 14 different types of cancer, and a test set of 54 patients. Gene expression measurements were available for 16,063 genes.

**DATA INFO.** One gene per row, one sample per column. Cancer classes are labelled as follows:

|     |           |     |        |     |        |
|----:|:----------|----:|:-------|----:|:-------|
|1.   |breast     |2.   |prostate|3.   |lung    |
|4.   |collerectal|5.   |lymphoma|6.   |bladder |
|7.   |melanoma   |8.   |uterus  |9.   |leukemia|
|10.  |renal      |11.  |pancreas|12.  |ovary   |
|13.  |meso       |14.  |cns     |

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, Normalizer
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

%matplotlib inline

# define commonly used colors
GRAY1, GRAY4, PURPLE = '#231F20', '#646369', '#A020F0'
BLUE, ORANGE, BLUE1 = '#57B5E8', '#E69E00', '#174A7E'
# configure plot font family to Arial
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.linewidth'] = 0.5

## Load and Prepare Data

In [2]:
data = np.load('../data/14cancer.npy.npz')['data']
# last column contains 'is test' flag
is_test = data[:, -1].astype(int)
data_test = data[is_test == 1, :]
data_train = data[is_test == 0, :]
# pre-last column contains class
y_train = data_train[:, -2].astype(int) - 1
y_test = data_test[:, -2].astype(int) - 1
X_train = data_train[:, :-2]
X_test = data_test[:, :-2]

# all 144 training samples are slitted into 8 CV folds
cv_indices = np.array([
    [5,   2,   1,   3,   6,   4,   7,   8],
    [14,  15,  12,  9,   11,  16,  10,  13],
    [23,  19,  20,  17,  21,  24,  18,  22],
    [31,  32,  29,  28,  26,  30,  25,  27],
    [35,  48,  38,  46,  42,  34,  47,  33],
    [44,  45,  41,  40,  37,  43,  39,  36],
    [55,  56,  49,  51,  53,  50,  52,  54],
    [63,  59,  64,  61,  60,  62,  57,  58],
    [69,  71,  67,  66,  72,  68,  70,  65],
    [87,  91,  76,  86,  81,  88,  83,  96],
    [92,  74,  89,  93,  95,  84,  79,  73],
    [85,  90,  75,  77,  82,  94,  80,  78],
    [99,  103, 98,  100, 97,  104, 102, 101],
    [105, 111, 106, 109, 107, 112, 108, 110],
    [117, 118, 120, 113, 116, 115, 119, 114],
    [128, 121, 122, 124, 125, 127, 123, 126],
    [133, 139, 137, 138, 132, 142, 144, 135],
    [136, 129, 130, 134, 141, 131, 143, 140]])

# in order to be used in GridSearchCV we need to reformat
# cv folds into the list of train-test indices
cv_indices = (cv_indices.T - 1).tolist()
cv_folds = []
for i in range(len(cv_indices)):
    train = [j for i in cv_indices[:i] + cv_indices[i + 1:] for j in i]
    cv_folds.append([train, cv_indices[i]])

## Cross-Validation
Let's write an auxilar function that calculates CV errors out of 144, its standard error and test errors out of 54. And one additional function for printing these values.

In [3]:
def calc_cv_stat(grid_search):
    cv_errors = 18*(1 - np.vstack(
        [grid_search.cv_results_[f'split{i}_test_score']
         for i in range(8)]).T)
    best_cv_errors = cv_errors[grid_search.best_index_, :]
    cv_errors_cnt = np.sum(best_cv_errors)
    cv_errors_cnt_std = np.sqrt(np.var(best_cv_errors, ddof=1)*8)
    test_errors_cnt = np.sum(
        grid_search.best_estimator_.predict(X_test) != y_test)
    return cv_errors_cnt, cv_errors_cnt_std, test_errors_cnt


def print_cv_stat(grid_search):
    cv_err, cv_err_std, test_err = calc_cv_stat(grid_search)
    print(grid_search.best_params_)
    print(f'   CV errors {cv_err} ({cv_err_std:.1f}), Test errors {test_err}')

## Nearest Shrunken Centroids
Let's implement nearest shrunken centroid model ourselves. The implementation is the same as proposed in the book, but the result for this task is different.

In [4]:
class ShrunkenCentroid(BaseEstimator, ClassifierMixin):
    """Nearest shrunken centroid classifier.
    Each class is represented by its centroid, with test samples classified to
    the class with the nearest centroid.
    Parameters
    ----------
    delta : float, optional (default = None)
        Delta for shrinking centroids to remove features.
    Attributes
    ----------
    classes_:
        All classes found in the training data set.
    centroids_ :
        Centroid of each class.
    overall_centroid_:
        The overall mean of each feature.
    priors_:
        Class prior probabilities.
    vars_:
        Pooled within-class variances of features.
    shrunken_centroids_:
        Shrunken centroid of each class.
    features_used_:
        The indices of features that are not shrunken to the overall centroid.
    """
    def __init__(self, delta: float = 0):
        self.delta = delta

    def fit(self, X: np.ndarray, y: np.array) -> 'ShrunkenCentroid':
        """
        Fit the ShrunkenCentroid model according to the given training data.
        Parameters
        ----------
        X :
            Training vector, where n_samples is the number of samples and
            n_features is the number of features.
        y :
            Target values (integers)
        """
        # relabel target values to start from zero
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
        self.classes_ = label_encoder.classes_
        N, p, K = *X.shape, self.classes_.size
        # calculated overall centroid and prior probabilities
        self.overall_centroid_ = np.mean(X, axis=0)
        _, counts = np.unique(y, return_counts=True)
        self.priors_ = counts / N
        # calculate correction coefficients for each class
        m = np.atleast_2d((1/counts - 1/N)**0.5).T
        self.centroids_ =\
            np.vstack([np.mean(X[y == k, :], axis=0) for k in range(K)])
        # pooled within-class variance and deviation of features
        self.vars_ = np.zeros(shape=p)
        for k in range(K):
            self.vars_ += np.sum((X[y == k, :]-self.centroids_[k])**2, axis=0)
        self.vars_ /= (N - K)
        stds = self.vars_ ** 0.5
        # calculate shrunken centroids
        distances = self.centroids_ - self.overall_centroid_
        mean_std = np.median(stds)
        t_stats = distances / (stds + mean_std) / m
        t_stats_shrunken =\
            np.sign(t_stats) * (np.abs(t_stats) - self.delta).clip(0)
        self.shrunken_centroids_ =\
            self.overall_centroid_ + m * (stds + mean_std) * t_stats_shrunken
        self.features_used_ = np.squeeze(np.argwhere(np.sum(np.abs(
            self.shrunken_centroids_ - self.overall_centroid_), axis=0) > 0))
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Perform classification on an array of test vectors X.
        The predicted class C for each sample in X is returned.
        Parameters
        ----------
        X : array, shape = [n_samples, n_features]
        Returns
        -------
        C : array, shape = [n_samples]
        """
        N, K = X.shape[0], self.classes_.size
        discriminators = np.zeros(shape=(N, K))
        for i in range(N):
            discriminators[i] = -np.sum(
                (self.shrunken_centroids_ - X[i])**2 / self.vars_, axis=1) +\
                                2*np.log(self.priors_)
        return self.classes_[np.argmax(discriminators, axis=1)]

In [5]:
shrunken_centroid_clf = Pipeline([
    ('norm', Normalizer()),
    ('classifier', ShrunkenCentroid())])

shrunken_centroid_gs = GridSearchCV(
    shrunken_centroid_clf,
    {'classifier__delta': np.linspace(0, 5, 10)},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
sc_n_genes = shrunken_centroid_gs.best_estimator_[1].features_used_.size
print_cv_stat(shrunken_centroid_gs)
print(f'   Number of Genes Used {sc_n_genes}')

{'classifier__delta': 2.7777777777777777}
   CV errors 34.0 (4.7), Test errors 17
   Number of Genes Used 5417


## L2 Penalized Discriminant Analysis
**PAGE 659.** The computational techniques discussed in this section apply to any method that ftis a linear model with quadratic regularization on the coefficients. That includes all the methods discussed in this section, and many more. When $p > N$, the computations can be carried out in an $N$-dimensional space, rather than $p$, via the singular value decomposition introduced in Section 14.5. Here is the geometric intuition: just like two points in three-dimensional space always lie on a line, $N$ points in $p$-dimensional space lie in an $(N-1)$-dimensional affine subspace.

Given then $N \times p$ data matrix $\mathbf{X}$, let $$\mathbf{X} = \mathbf{U}\mathbf{D}\mathbf{V}^T = \mathbf{R}\mathbf{V}^T$$
be the singular-value decomposition (SVD) of $\mathbf{X}$; that is, $\mathbf{V}$ is $p \times N$ with orthonormal columns, $\mathbf{U}$ is $N \times N$ orthogonal, and $\mathbf{D}$ a diagonal matrix with elements $d_1 \ge d_2 \ge d_N \ge 0$. The matrix $\mathbf{R}$ is $N \times N$, with rows $r_i^T$.

As a simple example, let's first consider the estimates from a ridge regression:
$$\hat{\beta} = (\mathbf{X}^T\mathbf{X} + \lambda \mathbf{I})^{-1}\mathbf{X}^T\mathbf{y}$$
Replacing $\mathbf{X}$ by $\mathbf{R}\mathbf{V}^T$ and after some further manipulations, this can be shown to equal
$$\hat{\beta} = \mathbf{V}(\mathbf{R}^T\mathbf{R} + \lambda \mathbf{I})^{-1}\mathbf{R}^T\mathbf{y}$$
(Exercise 18.4). Thus $\hat{\beta}=\mathbf{V}\hat{\theta}$, where $\hat{\theta}$ is the ridge-regression estimate using the $N$ observations $(r_i, y_i), i=1, 2, ..., N$. In other wrods, we can simply reduce the data matrix from $\mathbf{X}$ to $\mathbf{R}$, and work with the rows of $\mathbf{R}$. This trick reduces the computational costs from $O(p^3)$ to $O(pN^2)$ when $p > N$.

**PAGE 660.** Let $f^*(r_i) = \theta_0 + r_i^T\theta$ with $r_i$ defined in (18.13), and consider the pair of optimization problems:
$$(\hat{\beta}_0, \hat{\beta}) = \arg \min_{\beta_0, \beta \in \mathbb{R}^p}\sum_{i=1}^{N}L(y_i, \beta_0+x_i^T\beta) + \lambda \beta^T\beta$$
$$(\hat{\theta}_0, \hat{\theta}) = \arg \min_{\theta_0, \theta \in \mathbb{R}^N}\sum_{i=1}^{N}L(y_i, \theta_0+r_i^T\theta) + \lambda \theta^T\theta$$
Then the $\hat{\beta}_0 = \hat{\theta}_0$, and $\hat{\beta}=\mathbf{V}\hat{\theta}$.

The theorem says that we can simply replace the $p$ vectors $x_i$ by the $N$-vectors $r_i$, and perform our penalized fit as before, but with far fewer predictors. The $N$-vector solution $\hat{\theta}$ is then transformed back to the $p$-vector solution via a simple matrix multiplication. This result is part of the statistics folklore, and deserve to be known more widely - see Hastie and Tibshirani (2004) for further details.

In [6]:
class L2PenalizedDiscriminantAnalysis(BaseEstimator, ClassifierMixin):
    """L2 Penalized Discriminant Analysis
    A classifier for high-dimensional problems when p>>N. Uses transformed
    space to optimize calculations.
    Parameters
    ----------
    lam :
        How much the variance-covariance matrix should be shrunken towards
        its diagonal. Lower value indicates higher shrinkage and more penalty.
    """
    def __init__(self, lam=0.99):
        self.lam = lam

    def fit(self,
            X: np.ndarray,
            y: np.array) -> 'L2PenalizedDiscriminantAnalysis':
        """Fit L2PenalizedDiscriminantAnalysis model according to the given
           training data and parameters.
        Parameters
        ----------
        X :
            Training data.
        y :
            Target values.
        """
        # relabel target values to start from zero
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
        self.classes_ = label_encoder.classes_
        K, N, p = self.classes_.size, X.shape[0], X.shape[1]
        # calculate classes priors
        _, counts_elements = np.unique(y, return_counts=True)
        priors = counts_elements/y.shape[0]
        # transform the predictors space using SVD
        u, s, vh = np.linalg.svd(X, full_matrices=False)
        U, D, V = u, np.diag(s), vh.T
        R = U @ D
        # calculate means and the variance-covariance matrix
        means = [np.mean(R[y == i], axis=0) for i in range(K)]
        covs = np.zeros((N, N))
        for k in range(K):
            R_k = R[y == k, :]
            for i in range(R_k.shape[0]):
                xc = np.atleast_2d(R_k[i, :] - means[k]).T
                covs += (xc @ xc.T) / (N - K)
        covs = self.lam * covs + (1 - self.lam) * np.diag(np.diag(covs))
        covs_inv = np.linalg.inv(covs)
        # calculate intercepts and coefficients in the original space
        self.coef_, self.intercept_ = [], []
        for k in range(K):
            m_k = np.atleast_2d(means[k]).T
            p_k = priors[k]
            self.coef_.append(V @ covs_inv @ m_k)
            self.intercept_.append(-0.5 * m_k.T @ covs_inv @ m_k + np.log(p_k))
        self.intercept_ = np.hstack(self.intercept_)
        self.coef_ = np.hstack(self.coef_)
        return self

    def predict(self, X: np.ndarray) -> np.array:
        scores = X@self.coef_ + self.intercept_
        return self.classes_[np.argmax(scores, axis=1)]

In [7]:
penalized_discriminant_clf = Pipeline([
    ('norm', Normalizer()),
    ('classifier', L2PenalizedDiscriminantAnalysis())])

penalized_discriminant_gs = GridSearchCV(
    penalized_discriminant_clf,
    {'classifier__lam': np.linspace(0.1, 0.9, 9)},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
print_cv_stat(penalized_discriminant_gs)

{'classifier__lam': 0.8}
   CV errors 20.0 (3.0), Test errors 11


## Support Vector Classifier

In [8]:
support_vector_clf = Pipeline([
    ('norm', Normalizer()),
    ('classifier', LinearSVC(tol=1e-3))])

support_vector_gs = GridSearchCV(
    support_vector_clf,
    {'classifier__C': np.linspace(1000, 3000, 3)},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
print_cv_stat(support_vector_gs)

{'classifier__C': 1000.0}
   CV errors 27.0 (2.6), Test errors 14


## K-Nearest Neighbors

In [9]:
k_nearest_neighbors_clf = Pipeline([
    ('norm', Normalizer()),
    ('classifier', KNeighborsClassifier())])

k_nearest_neighbors_gs = GridSearchCV(
    k_nearest_neighbors_clf,
    {'classifier__n_neighbors': list(range(1, 5))},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
print_cv_stat(k_nearest_neighbors_gs)

{'classifier__n_neighbors': 1}
   CV errors 47.0 (3.8), Test errors 24


## L2 Penalized Multinomial

In [10]:
l2sgd = SGDClassifier(
    loss='log', penalty='l2', alpha=0.05, max_iter=10000, n_jobs=4,
    tol=1e-5, eta0=0.0005, learning_rate='adaptive', random_state=1)
l2_multinom_clf = Pipeline([
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('classifier', l2sgd)])
l2_multinom_gs = GridSearchCV(
    l2_multinom_clf,
    {'classifier__alpha': np.linspace(0.01, 0.03, 4)},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
print_cv_stat(l2_multinom_gs)

{'classifier__alpha': 0.016666666666666666}
   CV errors 24.0 (3.7), Test errors 16


## L1 Penalized Multinomial

In [11]:
l1sgd = SGDClassifier(
    loss='log', penalty='l1', alpha=0.05, max_iter=10000, n_jobs=4,
    tol=1e-5, eta0=0.0005, learning_rate='adaptive', random_state=1)
l1_multinom_clf = Pipeline([
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('classifier', l1sgd)])

# uncomment the line to run full grid search, it works realy slow
l1_multinom_gs = GridSearchCV(
    l1_multinom_clf,
    # {'classifier__alpha': np.linspace(0.03, 0.09, 7)},
    {'classifier__alpha': [0.05]},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
l1_multinom_n_genes = np.sum(np.sum(
    abs(l1_multinom_gs.best_estimator_[2].coef_), axis=0) != 0)
print_cv_stat(l1_multinom_gs)
print(f'   Number of Genes Used {l1_multinom_n_genes}')

{'classifier__alpha': 0.05}
   CV errors 23.000000000000004 (4.4), Test errors 13
   Number of Genes Used 278


## Elastic-net Penalized Multinomial

In [12]:
elastic_net_sgd = SGDClassifier(
    loss='log', penalty='elasticnet', alpha=0.08, l1_ratio=0.6, max_iter=10000,
    n_jobs=4, tol=1e-5, eta0=0.0005, learning_rate='adaptive', random_state=1)
elastic_net_clf = Pipeline([
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('classifier', elastic_net_sgd)])

# uncomment the line to run full grid search, it works realy slow
elastic_net_gs = GridSearchCV(
    elastic_net_clf,
    {'classifier__alpha': [0.08],  # [0.07, 0.08, 0.09],
     'classifier__l1_ratio': [0.6]},  # [0.5, 0.6, 0.7]},
    cv=cv_folds, iid=True, scoring='accuracy'
).fit(X_train, y_train)
elastic_net_n_genes = np.sum(np.sum(
    abs(elastic_net_gs.best_estimator_[2].coef_), axis=0) != 0)
print_cv_stat(elastic_net_gs)
print(f'   Number of Genes Used {elastic_net_n_genes}')

{'classifier__alpha': 0.08, 'classifier__l1_ratio': 0.6}
   CV errors 25.0 (5.5), Test errors 12
   Number of Genes Used 358


## SUMMARY

In [36]:
names = ['1. Nearest shrunken centroids',
         '2. L2-penalized discriminant|   analysis',
         '3. Support vector classifier',
         '5. k-nearest neighbors',
         '6. L2-penalized multinominal',
         '7. L1-penalized multinominal',
         '8. Elastic-net penalized|   multinominal']
grid_searches = [
    shrunken_centroid_gs, penalized_discriminant_gs, support_vector_gs,
    k_nearest_neighbors_gs, l2_multinom_gs, l1_multinom_gs, elastic_net_gs]
genes_used_cnt = [sc_n_genes, 16063, 16063, 16063, 16063, l1_multinom_n_genes,
                  elastic_net_n_genes]

In [37]:
# PAGE 656. TABLE 18.1. Prediction results for microarray data with 14 cancer
#           classes. Method 1 is described in Section 18.2. Method 2, 3 and 6
#           are discussed in Section 18.3, while 4, 7 and 8 are discussed in
#           Section 18.4. Method 5 is described in Section 13.3. The elastic-
#           net penalized multinomial does the best on the test data, but the
#           standard error of each test-error estimate is about 3, so such
#           comparisons are inconclusive.
print('Methods                        CV errors (SE)  Test errors   Number of')
print('                               Out of 144      out of 54    Genes Used')
print('----------------------------------------------------------------------')
for name, gs, genes_cnt in zip(names, grid_searches, genes_used_cnt):
    name = name.split('|')
    cv_err, cv_err_std, test_err = calc_cv_stat(gs)
    print(f'{name[0]:<29}{cv_err:>4.0f} ({cv_err_std:.1f}){test_err:>10}'
          f'{genes_cnt:>21,}')
    if len(name) > 1:
        print(name[1])

Methods                        CV errors (SE)  Test errors   Number of
                               Out of 144      out of 54    Genes Used
----------------------------------------------------------------------
1. Nearest shrunken centroids  34 (4.7)        17                5,417
2. L2-penalized discriminant   20 (3.0)        11               16,063
   analysis
3. Support vector classifier   27 (2.6)        14               16,063
5. k-nearest neighbors         47 (3.8)        24               16,063
6. L2-penalized multinominal   24 (3.7)        16               16,063
7. L1-penalized multinominal   23 (4.4)        13                  278
8. Elastic-net penalized       25 (5.5)        12                  358
   multinominal
